In [6]:
    from html.parser import HTMLParser
    from html.entities import name2codepoint
    import requests
    import os

    class MyHTMLParser(HTMLParser):
        
        def __init__(self, http, direct="", level=""):
            super().__init__()
            self._http = http
            self._lvl = level
            self._mode = None
            self._dir = direct
            self._fd = None
    
        def __delete__(self):
            if self._fd:
                close(self._fd)
    
        def run(self):
            print(f"{self._lvl}Running on {self._http}")
            resp = requests.get(self._http)
            self.feed(resp.content.decode('gbk'))
            
        def handle_starttag(self, tag, attrs):
            print(f"{self._lvl}Start tag:{tag} {len(self._lvl)}")
            if tag == "title":
                self._mode = f"t{len(self._lvl)}"
            elif tag == "p" and len(self._lvl) == 3:
                self._mode = "p3"
            elif tag == "br" and self._fd:
                self._fd.write(b'n')
    
            for attr in attrs:
            #   print("     attr:", attr)
                attr_type, val = attr
                """
                This part is highly dependent on the organizing structure of the website
                """
                if attr_type == 'href' and val.endswith('.htm') and val not in ['intro.htm', 'index.htm', '/index.htm']:
                    if self._http.endswith(".htm"):
                        bases = self._http.split('/')
                        base = '/'.join(bases[:-1])
                    else:
                        base = self._http
                    next_level = MyHTMLParser(base+"/"+val, self._dir, self._lvl+" ")
                    next_level.run()
             
        def handle_endtag(self, tag):
#            print(f"{self._lvl}End tag:{tag} {len(self._lvl)}")
            self._mode = None
    
        def handle_data(self, data):
    #        print(f"{self._lvl}Data     :", data)
            if self._mode in ["t1", "t2"]:
                if not self._dir:
                    self._dir = data
                else:
                    self._dir = os.path.join(self._dir, data)
                print(f"Create directory: {self._dir}")
                os.makedirs(self._dir)
                
            elif self._mode == "t3":
                file_path = os.path.join(self._dir, data)
                print(f"Create file: {file_path}")
                if not self._fd:
                    self._fd = open(file_path, "ab")
            elif self._mode == "p3":
                self._fd.write(bytes(data, "utf-8"))
                
        def handle_comment(self, data):
            print(f"{self._lvl}Comment  :", data)
    
        def handle_entityref(self, name):
            c = chr(name2codepoint[name])
            print(f"{self._lvl}Named ent:", c)
    
        def handle_charref(self, name):
            if name.startswith('x'):
                c = chr(int(name[1:], 16))
            else:
                c = chr(int(name))
            print(f"{self._lvl}Num ent  :", c)
    
        def handle_decl(self, data):
            print(f"{self._lvl}Decl     :", data)
    
    
    class MyHTMLParser1(MyHTMLParser):
    
        def __init__(self, http, direct="", level=""):
            super().__init__(http, direct, level)
                
        def run(self):
            print(f"{self._lvl}{len(self._lvl)}Running on {self._http}")
            resp = requests.get(self._http)
            decoded = resp.content.decode('utf-8')
            self.feed(decoded)
    
        def handle_starttag(self, tag, attrs):
            print(f"{self._lvl}Start tag:{tag} {len(self._lvl)}")
            if tag in ["a", "p"]:
                self._mode = f"{tag}{len(self._lvl)}"
            elif tag == 'br' and self._fd:
                self._fd.write('n')
    
            folder_name = None
            for attr in reversed(attrs):
                print("     attr:", attr)
                attr_type, val = attr
                if attr_type == "title":
                    folder_name = val
                elif attr_type == "href":
                    vals = val.split("/")
                    if self._mode == "a0" and folder_name and vals[0] == "" and vals[1] == "life-study-of-the-bible":
                        next_href = self._http + "/" + vals[2]
                        next_dir = os.path.join(self._dir, folder_name)
                        print(f"Creating dir {next_dir}")
                        os.makedirs(next_dir)
                        next_level = MyHTMLParser1(next_href, next_dir, self._lvl+" ")
                        next_level.run()
                    elif self._mode == "a1" and "/note/" not in val and "/#" not in val and "/life-study-of-the-bible/" in val and len(vals)>=4:
                        try:
                            x = int(vals[3])
                            next_href = self._http + "/" + vals[3]
                            next_level1 = MyHTMLParser1(next_href, os.path.join(self._dir, vals[3]), self._lvl+" ")
                            next_level1.run()
                        except ValueError as e:
                            pass
                        except Exception as e1:
                            print(f"EXCEPTION! {e1}")
    
        def handle_data(self, data):
    #        print(f"{self._lvl}Data     :", data)
            if self._mode in ["a2", "p2"]:
                if not self._fd:
                    self._fd = open(self._dir, "wt", encoding='utf-8')
                self._fd.write(data)
    
    
    class MyHTMLParser2(MyHTMLParser):
        def __init__(self, http, direct="", level=""):
            super().__init__(http, direct, level)
                
        def run(self):
#            print(f"{self._lvl}{len(self._lvl)}Running on {self._http}")
            if len(self._lvl) == 0:
                with open(self._http, encoding="utf-8") as fin:
                    decoded = fin.read()
            else:
                resp = requests.get(self._http)
                decoded = resp.content.decode('utf-8')
            self.feed(decoded)
    
        def handle_starttag(self, tag, attrs):
#            print(f"{self._lvl}Start tag:{tag} {len(self._lvl)}")
            if tag in ["a", "p", "title"]:
                self._mode = f"{tag}{len(self._lvl)}"
            elif tag == 'br' and self._fd:
                self._fd.write('n')
    
            folder_name = None
            for attr in reversed(attrs):
#                print("     attr:", attr)
                attr_type, val = attr
                vals = val.split("/") if isinstance(val, str) else []
                if attr_type == "href":
                    if self._mode == "a0" and "/book-" in val:
                        dirr = os.path.join(self._dir, vals[4])
                        if not os.path.exists(dirr):
                            next_level = MyHTMLParser2(val, self._dir, self._lvl+" ")
                            next_level.run()
                        else:
                            print(f"skipped ")
                    elif self._mode == "a1" and "/#" not in val and len(vals)>=4:
                        try:
                            x = int(vals[3])
                            next_href = self._http + vals[3]
                            dirr = os.path.join(self._dir, vals[2])
                            os.makedirs(dirr, exist_ok=True)
                            next_level1 = MyHTMLParser2(next_href, os.path.join(dirr, vals[3]), self._lvl+" ")
                            next_level1.run()
                        except ValueError as e:
                            pass
    
        def handle_data(self, data):
    #        print(f"{self._lvl}Data     :", data)
            if self._mode in ["a2", "p2"]:
                if not self._fd:
                    self._fd = open(self._dir, "wt", encoding='utf-8')
                self._fd.write(data)


In [ ]:
parser = MyHTMLParser2("C:\\Users\\klow55\\PycharmProjects\\crims2s\\notebooks\\full_listing.html.html", "NeeBooks")
parser.run()

In [5]:
x="https://bibleread.online/all-books-by-Watchman-Nee-and-Witness-Lee/book-administration-of-the-church-and-the-ministry-of-the-word-the-Witness-Lee-read-online"
x.split("/")[4]

'book-administration-of-the-church-and-the-ministry-of-the-word-the-Witness-Lee-read-online'